In [22]:
import pandas as pd

import numpy as np

import xlsxwriter, os

from collections import defaultdict

import matplotlib.pyplot as plt

%matplotlib inline

from openpyxl import load_workbook

from sklearn import preprocessing

In [98]:
# save sales by industry into excel files

def create_excel_fnc(path, df, sheetname):
    
    writer = pd.ExcelWriter(path)
    
    df.to_excel(writer, sheetname)
    
    writer.save()
    
def add_sheet_fnc(path, df,sheetname):
    
    book = load_workbook(path)
    
    writer = pd.ExcelWriter(path, engine='openpyxl')
    
    writer.book = book
    
    df.to_excel(writer, sheet_name=sheetname)
    
    writer.save()
    
    writer.close()

# Process MN Industry 2015 Dataset

In [38]:
# load MN Industry Dataset

MN_df = pd.read_excel(os.getcwd()+r'\MN_Data\MN_Industry2015.xls')

In [39]:
# drop unexpected rows

MN_df = MN_df[MN_df.COUNTY != 'NON-MINNESOTA CO']

In [41]:
# extract all industry names

Industry_Names = MN_df.INDUSTRY.unique()

In [53]:
County_Names = MN_df.COUNTY.unique().tolist()

In [113]:
# chosse one industry to do analysis by county

Industry_name = Industry_Names[2]

In [43]:
Food_df = MN_df[MN_df.INDUSTRY == '311 MFG -FOOD']

Food_df.head()

,YEAR,COUNTY,INDUSTRY,GROSS SALES,TAXABLE SALES,SALES TAX,USE TAX,TOTAL TAX,NUMBER
2,2015,AITKIN,311 MFG -FOOD,3502899,54989,3781,1,3782,4
42,2015,ANOKA,311 MFG -FOOD,7361521,1317029,90546,372,90918,15
113,2015,BECKER,311 MFG -FOOD,750425,15032,1034,187,1221,5
160,2015,BELTRAMI,311 MFG -FOOD,294666,235929,16221,206,16427,6
277,2015,BLUE EARTH,311 MFG -FOOD,1383165880,1501746,103245,19880,123125,8


In [54]:
Food_County = Food_df.COUNTY.tolist()       # Only 42 counties have food industry

Food_County_Cnt = len(Food_County)

In [56]:
# Extract Gross sales for county from food_df

Food_County_Sales = Food_df[['COUNTY', 'GROSS SALES']]

Food_County_Sales.set_index('COUNTY', inplace=True)

Food_County_Sales.head()

,GROSS SALES
COUNTY,
AITKIN,3502899
ANOKA,7361521
BECKER,750425
BELTRAMI,294666
BLUE EARTH,1383165880


In [88]:
# save food industry into excel

path1 = r'C:\Users\PL_Dell3668_One\Desktop\MN\Industry_Sales_Data\311 MFG -FOOD.xlsx'

create_excel_fnc(path1, Food_County_Sales, '311 MFG -FOOD.xlsx')

# Load our raw data by County

In [105]:
# load our data

Raw_Data = pd.ExcelFile(r'C:\Users\PL_Dell3668_One\Desktop\MN\Raw_Data.xlsx')

# Process Indices

In [101]:
def gen_indices_raw_data(path_raw, path_corr, sheetname, raw_data):
    
    df = raw_data.parse(sheetname)
    
    df.set_index('County', inplace=True)
    
    df = df.loc[Food_County]
    
    add_sheet_fnc(path_raw, df, sheetname)
    
    corr_df = pd.DataFrame()

    corr_df['Correlation Coefficient'] = df[df.columns[:]].apply(lambda x: x.corr(Food_County_Sales['GROSS SALES']))

    add_sheet_fnc(path_corr, corr_df, sheetname)

In [111]:
sheetnames = Raw_Data.sheet_names

indices_names = sheetnames[0:8]

In [112]:
path1 = r'C:\Users\PL_Dell3668_One\Desktop\MN\Industry_Raw_Data\Food\Indices.xlsx'

path2 = r'C:\Users\PL_Dell3668_One\Desktop\MN\Industry_Correlation_Data\Food\Indices_Corr.xlsx'

for sheetname in indices_names:
    
    gen_indices_raw_data(path1, path2, sheetname, Raw_Data)